# Cleaning

In this jupyter notebook a cleaning process is done on the dataframe explored and partly cleaned in 'code1_EDA'. The objectives are:
- Column name unification.

- Change how some columns show their data.

- Change the data type of some columns.

- Extreme values treatment.

- Null values treatment.

In [105]:
# Let's import the libraries that will be needed for this EDA.
import pandas as pd
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt
import sidetable
from datetime import date, datetime

In [106]:
# To show all the columns of our dataframe.
pd.options.display.max_columns=None

In [107]:
# Create the first dataframe containing the data from the csv obtained from Gbif.
df_0 = pd.read_csv('../data/mosquito1_explored.csv', index_col=0)

# Check the first three rows to see how this dataframe looks like.
df_0.head(3)

,gbifID,genus,species,taxonRank,countryCode,decimalLatitude,decimalLongitude,eventDate,day,month,year,taxonKey,identifiedBy,lastInterpreted,issue
0,4052772078,Aedes,Aedes albopictus,SPECIES,ES,41.51019,2.24589,2022-11-04T00:00:00,4,11,2022,1651430,Roger Eritja,2023-03-23T07:53:00.251Z,CONTINENT_DERIVED_FROM_COORDINATES
1,4052772077,Aedes,Aedes albopictus,SPECIES,IT,44.40289,8.98775,2021-08-27T00:00:00,27,8,2021,1651430,Karin Bakran-Lebl;Ana Klobucar;UNIROMA1;Roger ...,2023-03-23T07:53:02.575Z,CONTINENT_DERIVED_FROM_COORDINATES
2,4052772076,Culex,NaN,GENUS,NL,52.14539,5.39437,2021-09-25T00:00:00,25,9,2021,1497010,Hans-Peter Fuehrer;Mihaela Kavran;Karin Bakran...,2023-03-23T07:53:02.576Z,CONTINENT_DERIVED_FROM_COORDINATES


## 1. Change columns name

Most of the columns have their name in lower case letters but some have lower case mixed with capital letters.

In [108]:
# Create a dictionary to change just some columns names.
new_cols={'gbifID':'gbif_id','taxonRank':'taxon_rank','countryCode':'country_code','decimalLatitude':'latitude','decimalLongitude':'longitude','eventDate':'event_moment','taxonKey':'taxon_key','identifiedBy':'witness','lastInterpreted':'last_interpreted'}

# Change columns names.
df_0.rename(columns=new_cols,inplace=True)

# Check the first three rows to see how this dataframe looks like now.
df_0.head(3)

,gbif_id,genus,species,taxon_rank,country_code,latitude,longitude,event_moment,day,month,year,taxon_key,witness,last_interpreted,issue
0,4052772078,Aedes,Aedes albopictus,SPECIES,ES,41.51019,2.24589,2022-11-04T00:00:00,4,11,2022,1651430,Roger Eritja,2023-03-23T07:53:00.251Z,CONTINENT_DERIVED_FROM_COORDINATES
1,4052772077,Aedes,Aedes albopictus,SPECIES,IT,44.40289,8.98775,2021-08-27T00:00:00,27,8,2021,1651430,Karin Bakran-Lebl;Ana Klobucar;UNIROMA1;Roger ...,2023-03-23T07:53:02.575Z,CONTINENT_DERIVED_FROM_COORDINATES
2,4052772076,Culex,NaN,GENUS,NL,52.14539,5.39437,2021-09-25T00:00:00,25,9,2021,1497010,Hans-Peter Fuehrer;Mihaela Kavran;Karin Bakran...,2023-03-23T07:53:02.576Z,CONTINENT_DERIVED_FROM_COORDINATES


## 2. Not interesting variables

The objective of this project is to predict tiger mosquito (*Aedes albopictus*) so only this rows will be kept and in that way some columns can deleted.

In [109]:
# Create a new dataframe with only those rows with A. albopictus species.
df_1=df_0[df_0['species']=='Aedes albopictus']
df_1.head(3)

,gbif_id,genus,species,taxon_rank,country_code,latitude,longitude,event_moment,day,month,year,taxon_key,witness,last_interpreted,issue
0,4052772078,Aedes,Aedes albopictus,SPECIES,ES,41.51019,2.24589,2022-11-04T00:00:00,4,11,2022,1651430,Roger Eritja,2023-03-23T07:53:00.251Z,CONTINENT_DERIVED_FROM_COORDINATES
1,4052772077,Aedes,Aedes albopictus,SPECIES,IT,44.40289,8.98775,2021-08-27T00:00:00,27,8,2021,1651430,Karin Bakran-Lebl;Ana Klobucar;UNIROMA1;Roger ...,2023-03-23T07:53:02.575Z,CONTINENT_DERIVED_FROM_COORDINATES
3,4052772075,Aedes,Aedes albopictus,SPECIES,IT,41.70922,12.78512,2022-08-11T00:00:00,11,8,2022,1651430,UNIROMA1;Eleonora Longo;Francesco Severini;Rog...,2023-03-23T07:53:04.739Z,CONTINENT_DERIVED_FROM_COORDINATES


In [110]:
# Check dataframe shape.
df_1.shape

(14518, 15)

In [111]:
# Check unique values in 'species' column.
df_1['species'].value_counts()

Aedes albopictus    14518
Name: species, dtype: int64

In [112]:
# Check unique values in 'taxon_key' column.
df_1['taxon_key'].value_counts()

1651430    14518
Name: taxon_key, dtype: int64

These two columns have just one unique value and it appears 14518 times, the datafrae has the same number of rows.

In [113]:
# Check unique values in 'gbif_id' column.
df_1['gbif_id'].value_counts().sum()

14518

This column has 14518 unique values, the dataframe has the same number of rows.

In [114]:
# Check unique values in 'last_interpreted' column.
df_1['last_interpreted'].value_counts().sum()

14518

This column has 14518 unique values, the dataframe has the same number of rows. Moreover, this column contains the information about when the registration of this case of mosquito was analyzed, so this information is not important for this project's objective.

In [115]:
# Create a list with all the columns that are not interesting now.
columns_to_delete=['genus','species','taxon_rank','taxon_key','gbif_id','last_interpreted']
df_2=df_1.drop(columns_to_delete,axis=1,inplace=False)
df_2.head(3)

,country_code,latitude,longitude,event_moment,day,month,year,witness,issue
0,ES,41.51019,2.24589,2022-11-04T00:00:00,4,11,2022,Roger Eritja,CONTINENT_DERIVED_FROM_COORDINATES
1,IT,44.40289,8.98775,2021-08-27T00:00:00,27,8,2021,Karin Bakran-Lebl;Ana Klobucar;UNIROMA1;Roger ...,CONTINENT_DERIVED_FROM_COORDINATES
3,IT,41.70922,12.78512,2022-08-11T00:00:00,11,8,2022,UNIROMA1;Eleonora Longo;Francesco Severini;Rog...,CONTINENT_DERIVED_FROM_COORDINATES


## 3. Redundant variables

It was already checked if there were duplicated rows (there were not), but it is also important to check if there are columns that show the same information.

In [116]:
# Check how many unique values has each column to see if some have the same number of unique
#  values and check if they show the same information.
for col in df_2.columns:
    print(f'Column {col} has ---{len(df_2[col].unique())}--- unique values')

Column country_code has ---47--- unique values
Column latitude has ---11890--- unique values
Column longitude has ---12239--- unique values
Column event_moment has ---1776--- unique values
Column day has ---31--- unique values
Column month has ---12--- unique values
Column year has ---9--- unique values
Column witness has ---2034--- unique values
Column issue has ---2--- unique values


In [117]:
df_2.shape

(14518, 9)

The number of unique values that each column has is under 14518. Furthermore, each column has a different number of unique values.

Column 'event_date' has information about the mosquito registration time (date and time) and columns 'day', 'month' and 'year' has also information about this date, but all of them will be mantained.

## 4. Modify columns

Column 'event_moment' has date and time information, so this info must be separated.

In [118]:
# Divide the column 'event_moment' into 'event_date' and 'event_time'.
df_2['event_date']=df_2['event_moment'].str.split('T',expand=True, n=1)[0]
df_2['event_time']=df_2['event_moment'].str.split('T',expand=True, n=1)[1]
df_2.head(3)

,country_code,latitude,longitude,event_moment,day,month,year,witness,issue,event_date,event_time
0,ES,41.51019,2.24589,2022-11-04T00:00:00,4,11,2022,Roger Eritja,CONTINENT_DERIVED_FROM_COORDINATES,2022-11-04,00:00:00
1,IT,44.40289,8.98775,2021-08-27T00:00:00,27,8,2021,Karin Bakran-Lebl;Ana Klobucar;UNIROMA1;Roger ...,CONTINENT_DERIVED_FROM_COORDINATES,2021-08-27,00:00:00
3,IT,41.70922,12.78512,2022-08-11T00:00:00,11,8,2022,UNIROMA1;Eleonora Longo;Francesco Severini;Rog...,CONTINENT_DERIVED_FROM_COORDINATES,2022-08-11,00:00:00


In [119]:
# Check how many unique values there are in 'event_moment' column.
len(df_2['event_moment'].unique())

1776

In [120]:
# Check how many unique values there are in 'event_date' column.
len(df_2['event_date'].unique())

1776

In [121]:
# Check how many unique values there are in 'event_time' column.
df_2['event_time'].unique()

array(['00:00:00'], dtype=object)

There are the same values in 'event_moment' date as in 'event_date', and 'event_time' has only one unique value ('00:00:00', which gives no information) so 'event_moment' and 'event_time' can be deleted.

In [122]:
# Create a list with the columns that must be deleted.
columns_to_delete2=['event_moment','event_time']
df_3=df_2.drop(columns_to_delete2,axis=1,inplace=False)
df_3.head(3)

,country_code,latitude,longitude,day,month,year,witness,issue,event_date
0,ES,41.51019,2.24589,4,11,2022,Roger Eritja,CONTINENT_DERIVED_FROM_COORDINATES,2022-11-04
1,IT,44.40289,8.98775,27,8,2021,Karin Bakran-Lebl;Ana Klobucar;UNIROMA1;Roger ...,CONTINENT_DERIVED_FROM_COORDINATES,2021-08-27
3,IT,41.70922,12.78512,11,8,2022,UNIROMA1;Eleonora Longo;Francesco Severini;Rog...,CONTINENT_DERIVED_FROM_COORDINATES,2022-08-11


Now columns 'day', 'month' and 'year' must be joined to check if they contain the same information as 'event_date'.

In [123]:
# Join these three columns into one ('check_date').
df_3['check_date'] = df_3[['year','month','day']].apply(lambda x:'-'.join(str(value) for value in x), axis=1)
df_3.head(3)

,country_code,latitude,longitude,day,month,year,witness,issue,event_date,check_date
0,ES,41.51019,2.24589,4,11,2022,Roger Eritja,CONTINENT_DERIVED_FROM_COORDINATES,2022-11-04,2022-11-4
1,IT,44.40289,8.98775,27,8,2021,Karin Bakran-Lebl;Ana Klobucar;UNIROMA1;Roger ...,CONTINENT_DERIVED_FROM_COORDINATES,2021-08-27,2021-8-27
3,IT,41.70922,12.78512,11,8,2022,UNIROMA1;Eleonora Longo;Francesco Severini;Rog...,CONTINENT_DERIVED_FROM_COORDINATES,2022-08-11,2022-8-11


In [124]:
# 
df_3['new_check_date'] = pd.to_datetime(df_3['check_date'], format='%Y-%m-%d')
df_3.head(3)

,country_code,latitude,longitude,day,month,year,witness,issue,event_date,check_date,new_check_date
0,ES,41.51019,2.24589,4,11,2022,Roger Eritja,CONTINENT_DERIVED_FROM_COORDINATES,2022-11-04,2022-11-4,2022-11-04
1,IT,44.40289,8.98775,27,8,2021,Karin Bakran-Lebl;Ana Klobucar;UNIROMA1;Roger ...,CONTINENT_DERIVED_FROM_COORDINATES,2021-08-27,2021-8-27,2021-08-27
3,IT,41.70922,12.78512,11,8,2022,UNIROMA1;Eleonora Longo;Francesco Severini;Rog...,CONTINENT_DERIVED_FROM_COORDINATES,2022-08-11,2022-8-11,2022-08-11


In [126]:
df_3['new_event_date'] = pd.to_datetime(df_3['event_date'], format='%Y-%m-%d')
df_3.head(3)

,country_code,latitude,longitude,day,month,year,witness,issue,event_date,check_date,new_check_date,new_event_date
0,ES,41.51019,2.24589,4,11,2022,Roger Eritja,CONTINENT_DERIVED_FROM_COORDINATES,2022-11-04,2022-11-4,2022-11-04,2022-11-04
1,IT,44.40289,8.98775,27,8,2021,Karin Bakran-Lebl;Ana Klobucar;UNIROMA1;Roger ...,CONTINENT_DERIVED_FROM_COORDINATES,2021-08-27,2021-8-27,2021-08-27,2021-08-27
3,IT,41.70922,12.78512,11,8,2022,UNIROMA1;Eleonora Longo;Francesco Severini;Rog...,CONTINENT_DERIVED_FROM_COORDINATES,2022-08-11,2022-8-11,2022-08-11,2022-08-11


In [129]:
df_3.dtypes

country_code              object
latitude                 float64
longitude                float64
day                        int64
month                      int64
year                       int64
witness                   object
issue                     object
event_date                object
check_date                object
new_check_date    datetime64[ns]
new_event_date    datetime64[ns]
dtype: object

In [130]:
df_3['same_date'] = np.where(df_3['new_check_date']==df_3['new_event_date'], True, False)
df_3.head(3)

,country_code,latitude,longitude,day,month,year,witness,issue,event_date,check_date,new_check_date,new_event_date,same_date
0,ES,41.51019,2.24589,4,11,2022,Roger Eritja,CONTINENT_DERIVED_FROM_COORDINATES,2022-11-04,2022-11-4,2022-11-04,2022-11-04,True
1,IT,44.40289,8.98775,27,8,2021,Karin Bakran-Lebl;Ana Klobucar;UNIROMA1;Roger ...,CONTINENT_DERIVED_FROM_COORDINATES,2021-08-27,2021-8-27,2021-08-27,2021-08-27,True
3,IT,41.70922,12.78512,11,8,2022,UNIROMA1;Eleonora Longo;Francesco Severini;Rog...,CONTINENT_DERIVED_FROM_COORDINATES,2022-08-11,2022-8-11,2022-08-11,2022-08-11,True


In [133]:
df_3['same_date'].value_counts()

True    14518
Name: same_date, dtype: int64

In [ ]:
Nuestro df contiene las siguientes columnas:

registro = (instant), es el índice
fecha = (dteday)
estacion = (season) Hay que cambiarla, algunas estaciones no coinciden con la fecha.
año = (year) También hay que cambiarla, está en 0 y 1 (2018,2019)
mes = (month)
festivo = (holiday) 0: laborales, 1:festivos, creemos que tomará festivos.
dia_semana = (weekday)
no_laboral = (workingday) Cuenta como 0 el laboral y el 1 el no laboral
clima = ('weathersit')
'temperatura' = (temp)
'sens_termica'= ('atemp')
humedad = (hum)
viento = (windspeed)
ocasionales = (casual)
registrados = (registred)
total = (cnt)

In [ ]:
# Check again null values looking at its percentage.
df_1.stb.missing()

In [ ]:
# Check for outliers in month using another method.
mean_month=df_1['month'].mean()
std_month=df_1['month'].std()
upper=mean_month + std_month
lower=mean_month - std_month
ucb=mean_month + std_month * 3
lcb=mean_month - std_month * 3

In [ ]:
# Start the graph.
month_graph=sns.histplot(x=df_1['month'],kde=True)
graph.axvline(x=mean_month, c='red',label='mean')

# Plot its standard deviation.
graph.axvline(x=upper,c='green',label='std')
graph.axvline(x=lower, c='green')

# Plot its confidence interval to 99.7%
graph.axvline(x=lcb, c='orange',label='99 lower')
graph.axvline(x=ucb, c='orange',label='99 upper')

plt.legend()